In [1]:
from selenium import webdriver
import re
import json
from bs4 import BeautifulSoup

driver = webdriver.Chrome('/Users/sh/documents/tools/chromedriver')
loading = driver.implicitly_wait(3)

page_num = 0
origin_url = driver.get(
    f'https://www.airbnb.co.kr/s/서울특별시/homes?refinement_paths%5B%5D=%2Fhomes&allow_override%5B%5D=&map_toggle=true&s_tag=TibboRl0&section_offset={page_num}'
)

html = driver.page_source
soup = BeautifulSoup(html, 'lxml')
target = soup.select('div._1mpo9ida')
print(target)
while True:
    for pin in target:
        
        # 디테일 페이지 접속 
        rooms_id = pin.select_one('div._v72lrv > div > a:nth-of-type(1)').get('href')
        target_url = 'https://www.airbnb.co.kr'


        #         # 썸네일 이미지
        #         thumbnail_image_source = pin.select_one('div._1df8dftk').get('style')
        #         thumbnail_image = re.findall(r'\w*http\S*\w*jpg' , thumbnail_image_source)[0]

        #         # 숙소 이름
        #         rooms_name = pin.select_one('div._jnrahhr').get_text(strip=True)
        
        # 1박 가격
        price_source = pin.select_one('span._17oldnte > span:nth-of-type(2)').get_text(strip=True)
        price_parse = re.findall(r'[^₩\W]', price_source)
        price = ''.join(price_parse)
        
        print(thumbnail_image)
        print(rooms_name)
        print(price)
        
        driver.get(target_url + rooms_id)
        
        room_detail_html = driver.page_source
        room_detail_soup = BeautifulSoup(room_detail_html, 'lxml')
        room_detail_target = room_detail_soup.select('div#room')
        
        for detail_pin in room_detail_target:
            
            # json file
            bootstrap_data = re.search(r'data-hypernova-key="spaspabundlejs" data-hypernova-id=".*?">&lt;!--(.*?)--&gt;</script>', room_detail_html)
            bootstrap_json = json.loads(bootstrap_data.group(1))
            listing_dict = bootstrap_json['bootstrapData']['reduxData']['homePDP']['listingInfo']['listing']
            
            # host 정보
            rooms_host_id = listing_dict['user']['id']
            rooms_host_first_name = listing_dict['user']['host_name']
            rooms_host_profile_img = listing_dict['user']['profile_pic_path']
        
            # 디테일 페이지 커버 이미지
            room_detail_image_cover_source = detail_pin.select_one('div._30cuyx5').get('style')
            room_detail_image_cover = re.findall(r'\w*http\S*\w*jpg', room_detail_image_cover_source)[0]
            
            # 지역 테그
            location_tag = detail_pin.select_one('div._1hpgssa1 > div:nth-of-type(2) > div').get('data-location')
            
            # 숙박 인원
            rooms_personnel_source = detail_pin.select_one('div#summary > div > div > div:nth-of-type(2) > div > div:nth-of-type(1) > div > div:nth-of-type(2) > span').get_text(strip=True)
            rooms_personnel = re.findall('(\d)', rooms_personnel_source)[0]
            
            # 객실 수
            try:
                rooms_amount_source = detail_pin.select_one('div#summary > div > div > div:nth-of-type(2) > div > div:nth-of-type(2) > div > div:nth-of-type(2) > span').get_text(strip=True)
                rooms_amount = re.findall('(\d)', rooms_amount_source)[0]
            except:
                rooms_amount = 1
            
            # 샤워실 갯수
            rooms_bathroom_source = detail_pin.select_one('div#summary > div > div > div:nth-of-type(2) > div > div:nth-of-type(4) > div > div:nth-of-type(2) > span').get_text(strip=True)
            rooms_bathroom = re.findall('(\d)', rooms_bathroom_source)[0]
            
            # 침대 갯수
            rooms_bed_source = detail_pin.select_one('div#summary > div > div > div:nth-of-type(2) > div > div:nth-of-type(3) > div > div:nth-of-type(2) > span').get_text(strip=True)
            rooms_bed = re.findall('(\d)', rooms_bed_source)[0]
            
            # 숙소 개요
            rooms_discription = detail_pin.select_one('div#details > div > div > div').get_text(strip=True)

            
            # 편의 시설
            rooms_facilities_source = detail_pin.select('div._iq8x9is > div > div._qtix31 > div._ni9axhe ')
            rooms_facilities = []
            for facilities in rooms_facilities_source:
                if facilities.select_one('div._ncwphzu') == None:
                    continue
                else:
                    rooms_facilities.append(facilities.select_one('div._ncwphzu').get_text(strip=True))
                    
            # 숙소 이용규칙
            rooms_rules_source = detail_pin.select('div#house-rules > div > section > div > div._ncwphzu')
            rooms_rules = []
            for rules in rooms_rules_source:
                rooms_rules.append(rules.get_text(strip=True))
                
            # 주소 목록
            address_list = listing_dict['location_title'].split(', ')
            length = len(address_list)
            
            # 최소 예약 가능일
            minimum_check_in_duration = listing_dict['min_nights']
            
            # 주소
            country = address_list[length-1]
            city = address_list[length-2]
            district = listing_dict['localized_city']
            address1 = address_list[0] if length > 3 else ''

            # 위도, 경도
            lat = listing_dict['lat']
            lng = listing_dict['lng']

            print('host_id :', rooms_host_id)
            print('host_name:', rooms_host_first_name)
            print('host_profile_img:', rooms_host_profile_img)
            
            print(room_detail_image_cover)
            print(location_tag)
            print(rooms_personnel)
            print(rooms_amount)
            print(rooms_bathroom)
            print(rooms_bed)
            print(rooms_discription)
            print(rooms_facilities)
            print(rooms_rules)
            print(minimum_check_in_duration)
            
            print(country)
            print(city)
            print(district)
            print(address1)
            
            print(lat)
            print(lng)
            
            driver.execute_script("window.history.go(-1)") 

[<div class="_1mpo9ida"><div class="_gig1e7"><div itemprop="itemListElement" itemscope="" itemtype="http://schema.org/ListItem"><meta content="Mapo basecamp - undefined - 마포구" itemprop="name"/><meta content="1" itemprop="position"/><meta content="www.airbnb.co.kr/rooms/22320269?location=%EC%84%9C%EC%9A%B8%ED%8A%B9%EB%B3%84%EC%8B%9C" itemprop="url"/><div><div class="_f21qs6" id="listing-22320269"><div><div><div class="_1dp4576"><div><div class="_i2484c"><button aria-busy="false" class="_1rp5252" id="22320269" style="padding: 8px; margin: -8px;" type="button"><svg aria-label="목록에 숙소 추가하기" fill="#484848" fill-opacity="0.5" focusable="false" role="img" stroke="#ffffff" stroke-linecap="round" stroke-linejoin="round" stroke-width="2.5" style="height: 28px; width: 28px; display: block;" viewbox="0 0 32 32" xmlns="http://www.w3.org/2000/svg"><path d="m23.99 2.75c-.3 0-.6.02-.9.05-1.14.13-2.29.51-3.41 1.14-1.23.68-2.41 1.62-3.69 2.94-1.28-1.32-2.46-2.25-3.69-2.94-1.12-.62-2.27-1-3.41-1.14a7.96 

NameError: name 'thumbnail_image' is not defined